# One-Class Sampling as an Optimization Problem

$$
\begin{align}
    \text{SOP}\colon \! & \underset{\mathbf{v}, \mathbf{w}, \theta_\text{min}, \theta_\text{max}}{\text{minimize}} \quad \theta_\text{max} - \theta_\text{min}\\[1ex]
     \text{s.t.} & \sum_{j \in \mathcal{I}} v_j \! \cdot \! k(x_i, x_j) \geq  \theta_\text{min}, \, \forall i \in \mathcal{I}\\
     & \sum_{j \in \mathcal{I}} v_j \! \cdot \! k(x_i, x_j) \leq v_i \! \cdot \theta_\text{max}, \, \forall i \in \mathcal{I} \\
     & \sum_{j \in \mathcal{I}} w_i \! \cdot \! v_j \! \cdot \! k(x_i, x_j) \leq \theta_{\text{min}}, \, \forall i \in \mathcal{I} \\
     & \sum_{j \in \mathcal{I}} v_j > 0; \sum_{j \in \mathcal{I}} w_j = 1; \; v_j \geq w_j, \forall j \in \mathcal{I}\cup\mathcal{O} \\
     & v_j = 0, \; \forall j \in \mathcal{O}; v_j, w_j \in \{0,1\}, \; \forall j \in \{1, \dots, N\}
\end{align}
$$

Reformulated as a linear MIP problem:

$$
\begin{align}
    \text{SOP}\colon \! & \underset{\mathbf{v}, \mathbf{w}, \theta_\text{min}, \theta_\text{max}, \mu}{\text{minimize}} \quad \theta_\text{max} - \theta_\text{min}\\[1ex]
     \text{s.t.} & \sum_{j \in \mathcal{I}} v_j \! \cdot \! k(x_i, x_j) \geq  \color{red}{ v_i} \! \cdot \theta_\text{min}, \, \forall i \in \mathcal{I}\\
     & \sum_{j \in \mathcal{I}} v_j \! \cdot \! k(x_i, x_j) \leq v_i \! \cdot \theta_\text{max}, \, \forall i \in \mathcal{I} \\
     & \color{red}{\mu \geq \theta_\text{min}}\\
     & \color{red}{\sum_{j \in \mathcal{I}} v_j \! \cdot \! k(x_i, x_j) \geq \mu, \, \forall i \in \mathcal{I} }\\
     & \color{red}{\sum_{j \in \mathcal{I}} v_j \! \cdot \! k(x_i, x_j) - (N - 1)\! \cdot \!(1 - w_i)\leq \mu, \, \forall i \in \mathcal{I} }\\
     & \sum_{j \in \mathcal{I}} v_j > 0; \sum_{j \in \mathcal{I}} w_j = 1\\
     & v_j = 0, \; \forall j \in \mathcal{O}; v_j, w_j \in \{0,1\}, \; \forall j \in \{1, \dots, N\}
\end{align}
$$


In [ ]:
using Pkg
Pkg.activate("..")
using SVDD
using OneClassActiveLearning
using OneClassSampling
using MLKernels
using Random
using JuMP, Gurobi
using Printf
using Plots
pyplot()

Silent logging

In [ ]:
using Memento
Memento.config!(OneClassSampling.LOGGER, "warn"; fmt="[{level} | {name}]: {msg}")
Memento.config!(SVDD.LOGGER, "warn"; fmt="[{level} | {name}]: {msg}")

Load helpers

In [ ]:
include("../scripts/util/evaluate.jl")
function evaluate_with_svdd(model::DataType, init_strategy, solver,
                            data::Array{Float64, 2}, labels::Vector{Symbol},
                            test_data::Array{Float64, 2}, test_labels::Vector{Symbol},
                            quality_metrics)
    time_train = @elapsed model = train_svdd_model(model, init_strategy, solver, data, labels)
    time_pred = @elapsed pred = predict_svdd_model(model, test_data)
    scores = evaluate_prediction(quality_metrics, test_labels, pred)
    gamma = MLKernels.getvalue(model.kernel_fct.alpha)
    C = model.C
    num_support_vectors = length(SVDD.get_support_vectors(model))
    add_evaluation_stats!(scores, time_train, time_pred, gamma, C, num_support_vectors)
    return scores, model
end

In [ ]:
function ocs_visualize_2d(x, l, x_plot_axis, sample_mask, original_model, name=nothing, c=:red)
    grid = hcat([[x,y] for x in x_plot_axis for y in x_plot_axis]...)
    grid_pred = SVDD.predict(original_model, grid)
    f = contour(x_plot_axis, x_plot_axis, reshape(grid_pred , length(x_plot_axis), length(x_plot_axis)), levels=[0], linewidth=2, color=:grey, cbar=false, legend=false)
    if !all(sample_mask)
        scatter!(x[1, .!sample_mask], x[2, .!sample_mask], ms=5, color=:grey, markerstrokecolor=:grey)
    end

    if count(sample_mask) > 1
        scores, model = evaluate_with_svdd(VanillaSVDD, init_strat, solver, x[:, sample_mask], l, x, l, quality_metrics)        
        grid = hcat([[x,y] for x in x_plot_axis for y in x_plot_axis]...)
        grid_pred = SVDD.predict(model, grid)
        contour!(x_plot_axis, x_plot_axis, reshape(grid_pred , length(x_plot_axis), length(x_plot_axis)), levels=[0], linewidth=2, color=c, style=:dash, cbar=false, legend=false)
    end
    scatter!(x[1, sample_mask], x[2, sample_mask], marker=:diamond, ms=7, color=c)
    xlims!(-7, 9.5)
    ylims!(-10, 10.5)
    annotate!([(1.25, -9, Plots.text(name, :center, 16))])
    return scores, plot(f, xaxis=false, yaxis=false, grid=false, legend=false, size=(600, 250))
end

In [ ]:
function run_visualize_2d(s, x, l, x_plot_axis, title_ext, c=:red)
    sampling_failed = false
    sample_mask = falses(length(l))
    try
        Random.seed!(0)
        sample_mask = OneClassSampling.sample(s, x, l)
    catch e
        @show e
        sampling_failed = true
    end
    title_plot = @sprintf "%s (|S| = %d)" title_ext count(sample_mask)
    scores, p = ocs_visualize_2d(x, l, x_plot_axis, sample_mask, original_model, title_plot, c)
    pad = length(title_ext) > 5 ? "\t" : "\t\t"
    if sampling_failed
        stats = @sprintf "[%s]%s Sampling failed" title_ext pad
    else
        stats = @sprintf "[%s]%s Sample size %5d\tratio = %3.2f" title_ext pad count(sample_mask) (count(sample_mask) / length(l))
        if scores !== nothing
            stats *= @sprintf "\t#SVs = %5d" scores[:num_support_vectors]
        end
    end
    println(stats)
    return p
end

### Model Parameters

In [ ]:
gurobi_env = Gurobi.Env()
solver = with_optimizer(Gurobi.Optimizer, gurobi_env; OutputFlag=0, Threads=1)
quality_metrics = Dict(:mcc => matthews_corr, :kappa => cohens_kappa, :f1 => f1_score)

Generate Data

In [ ]:
Random.seed!(0)

r, noise, n = 5, 0.1, 100
x = hcat(randn(2, Int(n / 2)) * 2 .+ 3, randn(2, Int(n / 2)) * 1.5 .- 2)
l = fill(:inlier, n)

x_plot = hcat([[x, y] for x in range(-10, stop=12, length=100) for y in range(-10, stop=12, length=100)]...)
x_plot_axis = collect(range(-10, stop=12, length=100))

gamma = 0.025
init_strat = SVDD.SimpleCombinedStrategy(SVDD.FixedGammaStrategy(MLKernels.GaussianKernel(gamma)), SVDD.FixedCStrategy(1))

In [ ]:
_, original_model = evaluate_with_svdd(VanillaSVDD, init_strat, solver, x, l, x, l, quality_metrics);

In [ ]:
c = KDECache(x, gamma)
K = c.K

In [ ]:
N = size(x, 2)
OP = Model(solver)

@variable(OP, v[1:N], Bin)
@variable(OP, w[1:N], Bin)
@variable(OP, θ_h)
@variable(OP, θ_l)
@variable(OP, μ)
@objective(OP, Min, θ_h - θ_l)

@constraint(OP, mintheta[i = 1:N], sum(v[j] * K[i, j] for j in 1:N) >= v[i] * θ_l)
@constraint(OP, maxtheta[i = 1:N], sum(v[j] * K[i, j] for j in 1:N) <= v[i] * θ_h)

@constraint(OP, sum(v) >= 1)
@constraint(OP, sum(w) >= 1)
@constraint(OP, sum(w) <= 1)
@constraint(OP, μ >= θ_l)
@constraint(OP, minmu[i = 1:N], sum(v[j] * K[i, j] for j in 1:N) >= μ)
@constraint(OP, maxmu[i = 1:N], sum(v[j] * K[i, j] for j in 1:N) - (N - 1) * (1 - w[i]) <= μ)

JuMP.optimize!(OP)

if termination_status(OP) == MOI.OPTIMAL
    v_sol = convert(BitArray, round.(JuMP.value.(v), digits=0))
    opt=objective_value(OP)
else
    v_sol = fill(size(x,2), 1)
    opt = Inf
end

In [ ]:
sum(v_sol)

In [ ]:
_, p = ocs_visualize_2d(x, l, x_plot_axis, v_sol, original_model, "Test", :red)
p